In [58]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from datetime import timedelta

In [80]:
# ba_1y = yf.Ticker('BA').history(period='1y')
ba_1y = pd.read_csv('../DataFiles/amzn_1y.csv', index_col='Date', parse_dates=True)
# data = ba_1y[['Open', 'High', 'Low', 'Close']]
df = ba_1y[['Open', 'Close']]

In [81]:
df['daily_increase_pct'] = (df['Close'] - df['Open'])/df['Open']*100
df['next_day_change'] = (df['Close'] - df['Close'].shift(1))/df['Close'] *100
df['is_trend'] = np.sign(df['next_day_change']) * np.sign(df['next_day_change'].shift(1))

In [82]:
for i in range(0, len(df)):
    df.loc[df.index[i], 'tr_length'] =  df.loc[df.index[i-1], 'tr_length'] + 1 if df.loc[df.index[i], 'is_trend'] == 1 else 0

In [83]:
df['trend_end'] = 0
df['trend_reversal'] = 0

In [84]:
for i in range(0, len(df)):
    if df.loc[df.index[i-1], 'tr_length'] > 1 and df.loc[df.index[i], 'tr_length'] == 0:
        df.loc[df.index[i], 'trend_end'] = 1

In [85]:
for i in range(0, len(df)):
    try:
        if df.loc[df.index[i], 'trend_end'] == 1 and \
             df.loc[df.index[i+2], 'tr_length'] == 2 and \
                 np.sign(df.loc[df.index[i-1], 'next_day_change']) != np.sign(df.loc[df.index[i+2], 'next_day_change']):
            df.loc[df.index[i], 'trend_reversal'] = 1
    except:
        pass

In [86]:
df[df['trend_reversal'] == 1]

,Open,Close,daily_increase_pct,next_day_change,is_trend,tr_length,trend_end,trend_reversal
Date,,,,,,,,
2019-09-06,1838.22,1833.51,-0.256226,-0.393235,-1.0,0.0,1,1
2019-11-29,1817.78,1800.80,-0.934106,-0.983452,-1.0,0.0,1,1
2020-01-28,1840.50,1853.25,0.692747,1.344125,-1.0,0.0,1,1
2020-05-21,2500.00,2446.74,-2.130400,-2.092580,-1.0,0.0,1,1
2020-05-29,2415.94,2442.37,1.093984,1.689752,-1.0,0.0,1,1
2020-07-13,3251.06,3104.00,-4.523448,-3.092784,-1.0,0.0,1,1


In [95]:
d_ind = datetime.date(datetime.strptime('2020-05-29', '%Y-%m-%d') )

In [96]:
df[d_ind - timedelta(days=6):d_ind + timedelta(days=5)]

,Open,Close,daily_increase_pct,next_day_change,is_trend,tr_length,trend_end,trend_reversal
Date,,,,,,,,
2020-05-26,2458.00,2421.86,-1.470301,-0.620184,1.0,2.0,0,0
2020-05-27,2404.99,2410.39,0.224533,-0.475857,1.0,3.0,0,0
2020-05-28,2384.33,2401.10,0.703342,-0.386906,1.0,4.0,0,0
2020-05-29,2415.94,2442.37,1.093984,1.689752,-1.0,0.0,1,1
2020-06-01,2448.00,2471.04,0.941176,1.160240,1.0,1.0,0,0
2020-06-02,2467.00,2472.41,0.219295,0.055412,1.0,2.0,0,0
2020-06-03,2468.01,2478.40,0.420987,0.241688,1.0,3.0,0,0
